In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
from IPython.core.display import HTML
import re

In [2]:
URL = 'https://overthecap.com/contract-history/quarterback/'

response = requests.get(URL)

In [3]:
type(response)

requests.models.Response

In [4]:
response.status_code

200

In [5]:
requests.get('https://overthecap.com/contract-history/quarterback/')

<Response [200]>

In [6]:
response.text

'\ufeff<!DOCTYPE html><html lang="en-US"><head><link rel="preconnect" href="https://fonts.gstatic.com/" crossorigin /><script>WebFontConfig={google:{families:["Open Sans","Oswald","Lato&display=swap"]}};if ( typeof WebFont === "object" && typeof WebFont.load === "function" ) { WebFont.load( WebFontConfig ); }</script><script data-optimized="1" src="https://overthecap.com/wp-content/plugins/litespeed-cache/assets/js/webfontloader.min.js"></script> <title>NFL Quarterback Contract History | Over The Cap</title><meta name="description" content="Contract History | Over the Cap"><meta name="author" content="Overthecap.com"><meta name="robots" content="noarchive"><link rel="profile" href="//gmpg.org/xfn/11"><link rel="pingback" href="https://overthecap.com/xmlrpc.php"><meta name="viewport" content="width=device-width, initial-scale=1.0"><style type="text/css">.BUF{background-color:#00133F !important;;color:#EE2F2B !important;}.BUF-reverse{background-color:#EE2F2B !important;;color:#00133F !im

In [7]:
soup = BS(response.text)
type(soup)

bs4.BeautifulSoup

In [8]:
print(soup.prettify())

<html>
 <body>
  <p>
   ﻿
   <!DOCTYPE html>
  </p>
  <link crossorigin="" href="https://fonts.gstatic.com/" rel="preconnect"/>
  <script>
   WebFontConfig={google:{families:["Open Sans","Oswald","Lato&display=swap"]}};if ( typeof WebFont === "object" && typeof WebFont.load === "function" ) { WebFont.load( WebFontConfig ); }
  </script>
  <script data-optimized="1" src="https://overthecap.com/wp-content/plugins/litespeed-cache/assets/js/webfontloader.min.js">
  </script>
  <title>
   NFL Quarterback Contract History | Over The Cap
  </title>
  <meta content="Contract History | Over the Cap" name="description"/>
  <meta content="Overthecap.com" name="author"/>
  <meta content="noarchive" name="robots"/>
  <link href="//gmpg.org/xfn/11" rel="profile"/>
  <link href="https://overthecap.com/xmlrpc.php" rel="pingback"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <style type="text/css">
   .BUF{background-color:#00133F !important;;color:#EE2F2B !important;}.B

In [9]:
salary = soup.find_all('table', attrs={'class': 'position-table sortable'})
print('Number of tables on site: ', len(salary))

Number of tables on site:  1


In [11]:
table = salary[0]
body = table.find_all('tr')
head = body[0]
body_rows = body[1:]

#iterate through the head HTML code and make list of clean headings

headings = []
for item in head.find_all('th'): 
    item = (item.text).rstrip('\n')
    headings.append(item)
print(headings)

['Player', 'Team', 'Year Signed', 'Years', '', 'Value', 'APY', 'Guaranteed', '', 'APY as % Of Cap At Signing', '', 'Inflated Value', 'Inflated APY', 'Inflated Guaranteed']


In [12]:
#loop through the rest of the rows

all_rows = [] # will be a list for list for all rows
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will old entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        #append aa to row - note one row entry is being appended
        row.append(aa)
    # append one row to all_rows
    all_rows.append(row)

In [13]:
print(all_rows)

[['Aaron Rodgers', 'Packers', '2022', '5', '', '$150815000', '$50271667', '$150665000', '', '24.1%', '', '$150815000', '$50271667', '$150665000'], ['Josh Allen', 'Bills', '2021', '6', '', '$258000000', '$43000000', '$150000000', '', '23.6%', '', '$294332055', '$49055342', '$171123288'], ['Patrick Mahomes', 'Chiefs', '2020', '10', '', '$450000000', '$45000000', '$141000000', '', '22.7%', '', '$472704339', '$47270434', '$148114026'], ['Deshaun Watson', 'Browns', '2022', '5', '', '$230000000', '$46000000', '$230000000', '', '22.1%', '', '$230000000', '$46000000', '$230000000'], ['Dak Prescott', 'Cowboys', '2021', '4', '', '$160000000', '$40000000', '$126000000', '', '21.9%', '', '$182531507', '$45632877', '$143743562'], ['Deshaun Watson', 'Texans', '2020', '4', '', '$156000000', '$39000000', '$111894247', '', '19.7%', '', '$163870838', '$40967709', '$117539769'], ['Derek Carr', 'Raiders', '2022', '3', '', '$121422481', '$40474160', '$65300000', '', '19.4%', '', '$121422481', '$40474160', 

In [14]:
salary_qb = pd.DataFrame(data=all_rows, columns=headings)
salary_qb.head()

,Player,Team,Year Signed,Years,,Value,APY,Guaranteed,,APY as % Of Cap At Signing,,Inflated Value,Inflated APY,Inflated Guaranteed
0,Aaron Rodgers,Packers,2022,5,,$150815000,$50271667,$150665000,,24.1%,,$150815000,$50271667,$150665000
1,Josh Allen,Bills,2021,6,,$258000000,$43000000,$150000000,,23.6%,,$294332055,$49055342,$171123288
2,Patrick Mahomes,Chiefs,2020,10,,$450000000,$45000000,$141000000,,22.7%,,$472704339,$47270434,$148114026
3,Deshaun Watson,Browns,2022,5,,$230000000,$46000000,$230000000,,22.1%,,$230000000,$46000000,$230000000
4,Dak Prescott,Cowboys,2021,4,,$160000000,$40000000,$126000000,,21.9%,,$182531507,$45632877,$143743562


In [15]:
salary_qb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1404 entries, 0 to 1403
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Player                      1404 non-null   object
 1   Team                        1404 non-null   object
 2   Year Signed                 1404 non-null   object
 3   Years                       1404 non-null   object
 4                               1404 non-null   object
 5   Value                       1404 non-null   object
 6   APY                         1404 non-null   object
 7   Guaranteed                  1404 non-null   object
 8                               1404 non-null   object
 9   APY as % Of Cap At Signing  1404 non-null   object
 10                              1404 non-null   object
 11  Inflated Value              1404 non-null   object
 12  Inflated APY                1404 non-null   object
 13  Inflated Guaranteed         1404 non-null   obje

In [16]:
salary_qb['Value'] = salary_qb['Value'].str.replace('$', '')
salary_qb['APY'] = salary_qb['APY'].str.replace('$', '')
salary_qb['Guaranteed'] = salary_qb['Guaranteed'].str.replace('$', '')
salary_qb['Inflated Value'] = salary_qb['Inflated Value'].str.replace('$', '')
salary_qb['Inflated APY'] = salary_qb['Inflated APY'].str.replace('$', '')
salary_qb['Inflated Guaranteed'] = salary_qb['Inflated Guaranteed'].str.replace('$', '')

salary_qb.head()

/var/folders/1g/3d0xzz0j00j22wfb5pvjmw_m0000gp/T/ipykernel_81981/93601018.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  salary_qb['Value'] = salary_qb['Value'].str.replace('$', '')
/var/folders/1g/3d0xzz0j00j22wfb5pvjmw_m0000gp/T/ipykernel_81981/93601018.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  salary_qb['APY'] = salary_qb['APY'].str.replace('$', '')
/var/folders/1g/3d0xzz0j00j22wfb5pvjmw_m0000gp/T/ipykernel_81981/93601018.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  salary_qb['Guaranteed'] = sala

,Player,Team,Year Signed,Years,,Value,APY,Guaranteed,,APY as % Of Cap At Signing,,Inflated Value,Inflated APY,Inflated Guaranteed
0,Aaron Rodgers,Packers,2022,5,,150815000,50271667,150665000,,24.1%,,150815000,50271667,150665000
1,Josh Allen,Bills,2021,6,,258000000,43000000,150000000,,23.6%,,294332055,49055342,171123288
2,Patrick Mahomes,Chiefs,2020,10,,450000000,45000000,141000000,,22.7%,,472704339,47270434,148114026
3,Deshaun Watson,Browns,2022,5,,230000000,46000000,230000000,,22.1%,,230000000,46000000,230000000
4,Dak Prescott,Cowboys,2021,4,,160000000,40000000,126000000,,21.9%,,182531507,45632877,143743562


In [17]:
salary_qb['APY as % Of Cap At Signing'] = salary_qb['APY as % Of Cap At Signing'].str.replace('%', '')
salary_qb.head()

,Player,Team,Year Signed,Years,,Value,APY,Guaranteed,,APY as % Of Cap At Signing,,Inflated Value,Inflated APY,Inflated Guaranteed
0,Aaron Rodgers,Packers,2022,5,,150815000,50271667,150665000,,24.1,,150815000,50271667,150665000
1,Josh Allen,Bills,2021,6,,258000000,43000000,150000000,,23.6,,294332055,49055342,171123288
2,Patrick Mahomes,Chiefs,2020,10,,450000000,45000000,141000000,,22.7,,472704339,47270434,148114026
3,Deshaun Watson,Browns,2022,5,,230000000,46000000,230000000,,22.1,,230000000,46000000,230000000
4,Dak Prescott,Cowboys,2021,4,,160000000,40000000,126000000,,21.9,,182531507,45632877,143743562


In [26]:
salary_qb['Value'] = salary_qb['Value'].astype('float')
salary_qb['Years'] = salary_qb['Years'].astype('float')
salary_qb['APY'] = salary_qb['APY'].astype('float')
salary_qb['Guaranteed'] = salary_qb['Guaranteed'].astype('float')
salary_qb['APY as % Of Cap At Signing'] = salary_qb['APY as % Of Cap At Signing'].astype('float')
salary_qb['Inflated Value'] = salary_qb['Inflated Value'].astype('float')
salary_qb['Inflated APY'] = salary_qb['Inflated APY'].astype('float')
salary_qb['Inflated Guaranteed'] = salary_qb['Inflated Guaranteed'].astype('float')
salary_qb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1404 entries, 0 to 1403
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Player                      1404 non-null   object 
 1   Team                        1404 non-null   object 
 2   Year Signed                 1404 non-null   object 
 3   Years                       1404 non-null   float64
 4                               1404 non-null   object 
 5   Value                       1404 non-null   float64
 6   APY                         1404 non-null   float64
 7   Guaranteed                  1404 non-null   float64
 8                               1404 non-null   object 
 9   APY as % Of Cap At Signing  1404 non-null   float64
 10                              1404 non-null   object 
 11  Inflated Value              1404 non-null   float64
 12  Inflated APY                1404 non-null   float64
 13  Inflated Guaranteed         1404 

In [27]:
salary_qb.head()

,Player,Team,Year Signed,Years,,Value,APY,Guaranteed,,APY as % Of Cap At Signing,,Inflated Value,Inflated APY,Inflated Guaranteed
0,Aaron Rodgers,Packers,2022,5.0,,150815000.0,50271667.0,150665000.0,,24.1,,150815000.0,50271667.0,150665000.0
1,Josh Allen,Bills,2021,6.0,,258000000.0,43000000.0,150000000.0,,23.6,,294332055.0,49055342.0,171123288.0
2,Patrick Mahomes,Chiefs,2020,10.0,,450000000.0,45000000.0,141000000.0,,22.7,,472704339.0,47270434.0,148114026.0
3,Deshaun Watson,Browns,2022,5.0,,230000000.0,46000000.0,230000000.0,,22.1,,230000000.0,46000000.0,230000000.0
4,Dak Prescott,Cowboys,2021,4.0,,160000000.0,40000000.0,126000000.0,,21.9,,182531507.0,45632877.0,143743562.0


In [35]:
#salary_qb = salary_qb['Player', 'Team', 'Year Signed', 'Years', 'Value', 'APY', 'Guaranteed', 'APY as % Of Cap At Signing', 'Inflated Value', 'Inflated APY', 'Inflated Guaranteed']
salary_qb.drop([''], axis = 1, inplace = True)
salary_qb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1404 entries, 0 to 1403
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Player                      1404 non-null   object 
 1   Team                        1404 non-null   object 
 2   Year Signed                 1404 non-null   object 
 3   Years                       1404 non-null   float64
 4   Value                       1404 non-null   float64
 5   APY                         1404 non-null   float64
 6   Guaranteed                  1404 non-null   float64
 7   APY as % Of Cap At Signing  1404 non-null   float64
 8   Inflated Value              1404 non-null   float64
 9   Inflated APY                1404 non-null   float64
 10  Inflated Guaranteed         1404 non-null   float64
dtypes: float64(8), object(3)
memory usage: 120.8+ KB
